# Data cleaning + exploration

In [75]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from requests import get
from bs4 import BeautifulSoup
import numpy as np
import re

## 01. Load dataset including transition power

In [13]:
radio_dir = Path.home() / 'Desktop/insight_fellows/radio-project-tbd'

In [14]:
df = pd.read_csv(radio_dir / 'data/radio_list.csv')

In [16]:
df.Band.value_counts()

FM     538
AM      52
HD1     23
HD2     14
HD3     10
HD4      4
SW       1
Name: Band, dtype: int64

In [17]:
df = df[df.Band.isin(['FM', 'AM'])]

In [18]:
df['frequency'] = df['Freq.'].astype(str) + ' ' + df.Band
df.drop(['Freq.', 'Band', 'For.'], axis = 1, inplace = True)
df

,Location,Station Name,Call Sign,Power,P. Unit,frequency
0,Ajax,KX 96,CJKX-FM,50,kW,95.9 FM
4,Algonquin Park,Weatheradio,CJNK-FM,50,w,100.1 FM
5,Algonquin Park East,Weatheradio,CJNK-FM-1,22,w,101.3 FM
6,Algonquin Park East,Algonquin Park,CFOA-FM,10,w,102.7 FM
7,Algonquin Park West,Weatheradio,CJNK-FM-2,22,w,101.3 FM
...,...,...,...,...,...,...
635,Wingham,CKNX,CKNX,10/1,kW,920.0 AM
636,Woodstock,Hope FM,CJFH-FM,37,w,94.3 FM
637,Woodstock,Country 104,CKDK-FM,51,kW,103.9 FM
638,Woodstock,Heart FM,CIHR-FM,20,kW,104.7 FM


In [19]:
df = df.rename(columns = {'Location': 'city of licence', 'Freq.': 'frequency'})

In [20]:
df.dropna()

,city of licence,Station Name,Call Sign,Power,P. Unit,frequency
0,Ajax,KX 96,CJKX-FM,50,kW,95.9 FM
4,Algonquin Park,Weatheradio,CJNK-FM,50,w,100.1 FM
5,Algonquin Park East,Weatheradio,CJNK-FM-1,22,w,101.3 FM
6,Algonquin Park East,Algonquin Park,CFOA-FM,10,w,102.7 FM
7,Algonquin Park West,Weatheradio,CJNK-FM-2,22,w,101.3 FM
...,...,...,...,...,...,...
635,Wingham,CKNX,CKNX,10/1,kW,920.0 AM
636,Woodstock,Hope FM,CJFH-FM,37,w,94.3 FM
637,Woodstock,Country 104,CKDK-FM,51,kW,103.9 FM
638,Woodstock,Heart FM,CIHR-FM,20,kW,104.7 FM


How many unique stations do we have?

In [21]:
df['Station Name'].nunique()

297

## 02. Loading dataset including station genre ("format")

In [22]:
genre_db = pd.read_html('https://en.wikipedia.org/wiki/List_of_radio_stations_in_Ontario', converters = {'Frequency': str})[0]

In [23]:
genre_db

,Call sign,Frequency,City of licence,Owner,Format
0,CJKX-FM,NaN,Ajax,Durham Radio,country
1,CKON-FM,NaN,Akwesasne,Akwesasne Communication Society,First Nations community radio
2,CJNK-FM,NaN,Algonquin Park,Weatheradio Canada,weather alerts
3,CJNK-FM-1,NaN,Algonquin Park East,Weatheradio Canada,weather alerts
4,CJNK-FM-2,NaN,Algonquin Park West,Weatheradio Canada,weather alerts
...,...,...,...,...,...
589,CJFH-FM,NaN,Woodstock,Sound of Faith Broadcasting,Christian radio
590,CKDK-FM,NaN,Woodstock,Corus Entertainment,country
591,CIHR-FM,NaN,Woodstock,Byrnes Communications,adult contemporary
592,CHPM-FM,NaN,Wunnummin Lake,Wawatay,First Nations community radio


In [24]:
soup = BeautifulSoup(get('https://en.wikipedia.org/wiki/List_of_radio_stations_in_Ontario').text, 'html.parser')

In [25]:
table = soup.find('table')
freq_index = list(genre_db.columns).index('Frequency')
city_index = list(genre_db.columns).index('City of licence')
genre_db['lat'] = np.nan
genre_db['long'] = np.nan
lat_index = list(genre_db.columns).index('lat')
long_index = list(genre_db.columns).index('long')

for row_n, row in enumerate(table.findAll('tr')[1:]):
    all_td = row.findAll('td')
    
    # add frequency
    frequency = all_td[freq_index].text.strip('0')
    genre_db.iloc[row_n, freq_index] = frequency
    
    # add coordinates
    try:
        city = all_td[city_index].find('a')['href']
        city_href = 'https://en.wikipedia.org' + city
        city_soup = BeautifulSoup(get(city_href).text, 'html.parser')
        coordinates = city_soup.find('span', 'geo').text.split(';')
        coordinates_stripped = [x.strip() for x in coordinates]
        genre_db.iloc[row_n, lat_index] = float(coordinates_stripped[0])
        genre_db.iloc[row_n, long_index] = float(coordinates_stripped[1])
    except:
        print('Did not get coordinates for', city)

Did not get coordinates for /wiki/Pigeon_River_Bridge


In [26]:
genre_db.columns = [column.lower() for column in genre_db.columns]
df.columns = [column.lower() for column in df.columns]

Any non-FM/AM radios in this dataset?

In [37]:
genre_db.frequency.str.split(' ', expand = True).iloc[:,1].unique()

array(['FM', 'AM'], dtype=object)

In [15]:
genre_db = genre_db.dropna()

In [16]:
genre_db['call sign'].nunique()

587

## 03. Merge power + genre datasets

In [38]:
new_df = genre_db.merge(df, how = 'outer', on = ['call sign']).reset_index(drop=True)

In [39]:
new_df['frequency'] = new_df.apply(lambda row: row.frequency_y if pd.isna(row.frequency_x) else row.frequency_x, axis = 1)
new_df['city of licence'] = new_df.apply(lambda row: row['city of licence_y'] if pd.isna(row['city of licence_x']) else row['city of licence_x'], axis = 1)

In [40]:
new_df.drop(['frequency_x', 'frequency_y', 'city of licence_x', 'city of licence_y'], axis = 1, inplace = True)

In [41]:
new_df.dropna(inplace=True)

## 04. Save merged dataframe

In [42]:
new_df.to_csv(radio_dir / 'data/radio_list_preprocessed.csv')

In [43]:
new_df

,call sign,owner,format,lat,long,station name,power,p. unit,frequency,city of licence
0,CJKX-FM,Durham Radio,country,43.85833,-79.03639,KX 96,50,kW,95.9 FM,Ajax
2,CJNK-FM,Weatheradio Canada,weather alerts,45.58417,-78.35833,Weatheradio,50,w,100.1 FM,Algonquin Park
3,CJNK-FM-1,Weatheradio Canada,weather alerts,45.58417,-78.35833,Weatheradio,22,w,101.3 FM,Algonquin Park East
4,CJNK-FM-2,Weatheradio Canada,weather alerts,45.58417,-78.35833,Weatheradio,22,w,101.3 FM,Algonquin Park West
5,CFOA-FM,Friends of Algonquin Park,tourist/park information,45.58417,-78.35833,Algonquin Park,10,w,102.7 FM,Algonquin Park
...,...,...,...,...,...,...,...,...,...,...
590,CKNX-FM,Blackburn Radio,adult contemporary,43.88778,-81.31167,101.7 The One,100,kW,101.7 FM,Wingham
591,CJFH-FM,Sound of Faith Broadcasting,Christian radio,43.13056,-80.74667,Hope FM,37,w,94.3 FM,Woodstock
592,CKDK-FM,Corus Entertainment,country,43.13056,-80.74667,Country 104,51,kW,103.9 FM,Woodstock
593,CIHR-FM,Byrnes Communications,adult contemporary,43.13056,-80.74667,Heart FM,20,kW,104.7 FM,Woodstock


## 05. EDA 🛠

In [45]:
station_counts = new_df['city of licence'].value_counts().reset_index()
station_counts.columns = ['city of licence', 'n_stations']
station_counts

,city of licence,n_stations
0,Toronto,36
1,Ottawa,26
2,Windsor,14
3,London,13
4,Greater Sudbury,13
...,...,...
198,Schreiber,1
199,Sioux Narrows,1
200,Port Hope,1
201,Kaministiquia,1


In [46]:
new_df = new_df.merge(station_counts, how = 'left', on = ['city of licence']).reset_index(drop=True)

In [60]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scattergeo(
        lon = new_df['long'],
        lat = new_df['lat'],
        text = new_df['city of licence'] + '\n' +
               new_df['n_stations'].astype(str) + ' station(s)',
        mode = 'markers',
        marker_size = new_df['n_stations'],
        ))

fig.update_layout(
        title = 'Radio station counts by location in Ontario',
        #geo_scope='north america',
        geo = dict(
            # adjusted for Ontario
            lataxis = dict(range=[40,60]),
            lonaxis = dict(range=[-100,-60])),
    )
fig.show()

In [61]:
new_df.format.nunique()

57

In [62]:
new_df.format.value_counts()

public news/talk                                    74
adult contemporary                                  61
First Nations community radio                       54
public news/talk (French)                           36
Christian radio                                     35
country                                             31
hot adult contemporary                              22
community radio                                     21
campus radio                                        19
classic hits                                        19
active rock                                         13
multilingual                                        13
public music                                        12
CHR                                                 11
tourist information                                  9
oldies                                               9
adult hits                                           9
news/talk                                            9
community 

In [80]:
languages = new_df.format.str.split(' \(', expand = True)[1]

In [88]:
languages = languages.apply(lambda x: x.strip(')') if x is not None else 'English')

In [89]:
languages.unique()

array(['English', 'German', 'French', 'Spanish'], dtype=object)

In [90]:
new_df['language'] = languages

In [95]:
new_df.to_csv(radio_dir / 'data/radio_list_preprocessed_wLanguages.csv')